In [66]:
import json

In [67]:
index = 4
version_num = "fig4_v14_shanto"
comp_id = "Sabrina"  # Replace with actual computer name

## Function to find g, anharmonicity, qubit frequency

In [68]:
import numpy as np
import scqubits as scq

# Constants
e = 1.602e-19  # elementary charge in C
hbar = 1.054e-34  # reduced Planck constant in Js
Z_0 = 50  # in Ohms

def find_g_a(C_g, C_B, f_r, Lj, N):
    C_Sigma = C_g + C_B # + 1.5e-15
    omega_r = 2 * np.pi * f_r
    EJ = ((hbar / 2 / e) ** 2) / Lj * (1.5092e24) # 1J = 1.5092e24 GHz
    EC = e**2/(2*C_Sigma) * (1.5092e24) # 1J = 1.5092e24 GHz

    transmon = scq.Transmon(EJ=EJ,
                            EC=EC,
                            ng = 0,
                            ncut = 30)

    a = transmon.anharmonicity() * 1000 # linear MHz
    g = ((C_g / C_Sigma) * omega_r * np.sqrt(N * Z_0 * e**2 / (hbar * np.pi) )* (EJ/(8*EC))**(1/4)) / 1E6 / (2 * np.pi) # linear MHz
    f_q = transmon.E01() # Linear GHz
    
    return g, a, f_q

### Sanity check with a known qubit (to make sure my code still works)

In [69]:
# Qubit Identifier 1

C_g = 5.91536e-15	# in F
C_B = 122.44002e-15	# in F
f_r = 6.116e9 # linear frequency in Hz
f_q = 4.216 # Linear freq in GHz
Lj = 9.686e-9
anharmonicity = -0.153 # Linear freq in GHz


print('Qubit 1')
print("Experimental g = 61 Linear MHz")
# print(f"Simulated g = {g(C_g, C_B, f_r, f_q, anharmonicity, 4) / 1E6 / (2 * np.pi)} Linear MHz")
print(f"Simulated g = {find_g_a(C_g, C_B, f_r, Lj, 4)[0]} Linear MHz")
print(f"Simulated a = {find_g_a(C_g, C_B, f_r, Lj, 4)[1]} Linear MHz")
print(f"Simulated f_q = {find_g_a(C_g, C_B, f_r, Lj, 4)[2]} Linear GHz")

Qubit 1
Experimental g = 61 Linear MHz
Simulated g = 67.84409429174482 Linear MHz
Simulated a = -164.35364443213808 Linear MHz
Simulated f_q = 4.354936065065626 Linear GHz


### Load LOM json (worth checking if the filename matches the qubit index)

In [70]:
lom_json = open(f"qubitonly_num{index}_{comp_id}_v{version_num}.json")
print(f"qubitonly_num{index}_v{version_num}")
      
data_lom = json.load(lom_json)

print(data_lom["data"])

qubitonly_num4_vfig4_v14_shanto
{'cross_to_ground': -140.93898, 'claw_to_ground': -45.88039, 'cross_to_claw': -1.83687, 'cross_to_cross': 144.99242, 'claw_to_claw': 47.92219, 'ground_to_ground': 244.39501}


### Load Eigenmode json (unfortunately you're gonna have to paste in the custom filename here)

In [71]:
import json

In [72]:
emode_json = open(f"cavity_claw_num{index}_{comp_id}_v{version_num}.json") # PASTE IN CUSTOM FILENAME

data_emode = json.load(emode_json)

# print(data_emode["design_options"]["geometry_dict"]["claw_opts"]["hfss_inductance"][0])
print(data_emode["sim_results"])

{'cavity_frequency': 6406539487.07, 'Q': 10881973.129265117, 'kappa': 588.729581572}


## Generate data (np.dataframe) using above method

In [73]:
data = {}

cross2cpw = abs(data_lom["data"]["cross_to_claw"]) * 1e-15
cross2ground = abs(data_lom["data"]["cross_to_ground"]) * 1e-15
f_r = data_emode["sim_results"]["cavity_frequency"]
Lj = data_lom["design_options"]["hfss_inductance"] * 1e-9
# print(Lj)
gg, aa, ff_q = find_g_a(cross2cpw, cross2ground, f_r, Lj, N=4)
data.update({f"qubit_{index}": dict(
    f_cavity = f_r,
    Q = data_emode["sim_results"]["Q"],
    kappa = data_emode["sim_results"]["kappa"],
    g = gg,
    a = aa,
    f_qubit = ff_q
)})

print(data)


{'qubit_4': {'f_cavity': 6406539487.07, 'Q': 10881973.129265117, 'kappa': 588.729581572, 'g': 20.566238250861083, 'a': -146.76090752045567, 'f_qubit': 4.21815709401853}}


In [74]:
data

{'qubit_4': {'f_cavity': 6406539487.07,
  'Q': 10881973.129265117,
  'kappa': 588.729581572,
  'g': 20.566238250861083,
  'a': -146.76090752045567,
  'f_qubit': 4.21815709401853}}

### Export to json (if desired)

In [75]:
from utils import *
save_simulation_data_to_json(data, f"final_q{index}_{comp_id}_v{version_num}")